In [14]:
"""
This module contains a collection of classes for generating input features to fit machine learning models to.
"""

import multiprocessing
import os
import logging

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures as SklearnPolynomialFeatures

import pymatgen
from pymatgen import Element, Composition
from pymatgen.ext.matproj import MPRester
#from citrination_client import CitrinationClient, PifQuery, SystemQuery, ChemicalFieldQuery, ChemicalFilter
# trouble? try: `pip install citrination_client=="2.1.0"`

# locate path to directory containing AtomicNumber.table, AtomicRadii.table AtomicVolume.table, etc
# (needs to do it the hard way becuase python -m sets cwd to wherever python is ran from)

# is this how you do it? Any other imports needed for this module then?
import inspect # used to get a dictionary of classes in a module
from matminer.featurizers import structure as st


import mastml
from mastml import utils
log = logging.getLogger('mastml')
print('mastml dir: ', mastml.__path__)
MAGPIE_DATA_PATH = os.path.join(mastml.__path__[0], '../magpie/')


class Matminer(BaseEstimator, TransformerMixin):
    def __init__(self, structural_features, structure_col): # _instantiate only needs this
        # assuming dataframe is coming in with a column 'Structure' with coords.
        # where do I need to raise errors
        structural_features = structural_features.split(', ') # structural feature is now cast as a list
        self.structural_features = structural_features # structural feature field of class
        self.structure_col = structure_col

    def fit(self, df, y=None):
        return self

    def transform(self, df, y=None):
        # any possible errors i need to raise?
        for struc_feat in range(len(self.structural_features)): # nested for loop to iterate through structural features list
          for feature_name in inspect.getmembers(structure,
                                               inspect.isclass):  # will get list of all classes in structure module

              if (feature_name[0] is self.structural_features[struc_feat]):
                  sf = getattr(structure, self.structural_features[struc_feat])() # instantiates the structure featurizer
                  df = sf.fit_featurize_dataframe(df, self.structure_col)  # fit_featurize_dataframe() works for all
                  log.info("BITCH RICH BITCH RICH")
                  print("made it")

        # if for loop passed, structural_feature was not a valid structural feature

        return df  # I think it should return the updated dataframe to concatenate

mastml dir:  ['/Users/AlexDo/PycharmProjects/MAST-ML/mastml']


In [5]:
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

STRUCTURE_DIR = '/Users/AlexDo/PycharmProjects/VASP_tools'

f = Poscar.from_file(STRUCTURE_DIR+'/'+'GaAs_POSCAR.POSCAR.vasp')
elements = f.site_symbols
structure = f.structure

df = pd.DataFrame()

d = {'Material': 'GaAs', 'Structure': [structure]}

df = pd.DataFrame(d)
df

,Material,Structure
0,GaAs,"[[0. 0. 0.] Ga, [0. 2.87509131 2.87509..."


In [6]:
inspect.getmembers(st, inspect.isclass)[0][0]


'AverageBondAngle'

In [9]:
matminer = Matminer('DensityFeatures', 'Structure')
matminer.__class__.__name__

'Matminer'

In [10]:
import argparse
import inspect
import os
import shutil
import logging
import warnings
from datetime import datetime
from collections import OrderedDict
from os.path import join # We use join tons
from functools import reduce

import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import make_scorer

from mastml import conf_parser, data_loader, html_helper, plot_helper, utils, learning_curve, data_cleaner, metrics
from mastml.legos import (data_splitters, feature_generators, feature_normalizers,
                    feature_selectors, model_finder, util_legos, randomizers, hyper_opt)
from mastml.legos import clusterers as legos_clusterers

log = logging.getLogger('mastml')

y = None
df1 = matminer.fit_transform(df, y)

df1

mastml dir:  ['/Users/AlexDo/PycharmProjects/MAST-ML/mastml']


,Material,Structure
0,GaAs,"[[0. 0. 0.] Ga, [0. 2.87509131 2.87509..."


In [15]:
matminer.fit_transform(df, y)

,Material,Structure
0,GaAs,"[[0. 0. 0.] Ga, [0. 2.87509131 2.87509..."


In [75]:
matminer('ElectronicRadialDistribution').fit_transform(df, y)

,Material,Structure,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,dimensionality
0,GaAs,"[[0. 0. 0.] Ga, [0. 2.87509131 2.87509...",216,cubic,1,False,3


In [76]:
matminer('JarvisCFID').fit_transform(df, y)

,Material,Structure,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,dimensionality,jml_bp_mult_atom_rad,jml_hfus_add_bp,jml_elec_aff_mult_voro_coord,...,jml_nn_91,jml_nn_92,jml_nn_93,jml_nn_94,jml_nn_95,jml_nn_96,jml_nn_97,jml_nn_98,jml_nn_99,jml_nn_100
0,GaAs,"[[0. 0. 0.] Ga, [0. 2.87509131 2.87509...",216,cubic,1,False,3,2120.075,1682.150125,2.6725,...,24.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,8.0


In [77]:
matminer('DensityFeatures').fit_transform(df, y)

,Material,Structure,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,dimensionality,jml_bp_mult_atom_rad,jml_hfus_add_bp,jml_elec_aff_mult_voro_coord,...,jml_nn_94,jml_nn_95,jml_nn_96,jml_nn_97,jml_nn_98,jml_nn_99,jml_nn_100,density,vpa,packing fraction
0,GaAs,"[[0. 0. 0.] Ga, [0. 2.87509131 2.87509...",216,cubic,1,False,3,2120.075,1682.150125,2.6725,...,0.0,12.0,0.0,0.0,0.0,0.0,8.0,5.053199,23.765936,0.327641


In [96]:
matminer('SOAP').fit_transform(df, y)

,Material,Structure,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,dimensionality,jml_bp_mult_atom_rad,jml_hfus_add_bp,jml_elec_aff_mult_voro_coord,...,xrd_118,xrd_119,xrd_120,xrd_121,xrd_122,xrd_123,xrd_124,xrd_125,xrd_126,xrd_127
0,GaAs,"[[0. 0. 0.] Ga, [0. 2.87509131 2.87509...",216,cubic,1,False,3,2120.075,1682.150125,2.6725,...,0.005027,0.001625,0.000972,0.002412,0.005063,0.006674,0.00546,0.00277,0.000872,0.00017


In [1]:
ht = 'Jar'

ht.split()

['Jar']

In [2]:
ht = 'Jar'

ht.split(',')

['Jar']

In [4]:
ht = 'Jar, Bar, Tar, Car, Far'

ht.split(', ')

['Jar', 'Bar', 'Tar', 'Car', 'Far']